In [1]:
#!pip install nltk
#!pip install -U scikit-learn

In [2]:
import pandas as pd
import numpy as np
import copy

from pymystem3 import Mystem

m = Mystem()

to_replace = ['Конверсии',
    'Конверсия (%)',
    'Цена цели (руб.)',
    'Ср. позиция кликов',
    'Ср. позиция показов',
    'Ср. цена клика (руб.)',
    'Расход (руб.)']

column_to_aggregate = {'Показы': 'sum',
                       'Клики': 'sum',
                       'Конверсии':'sum',
                       'Расход (руб.)':'sum'}

In [3]:
df = pd.read_csv('source/1.csv',skiprows=5,sep=';')

# Меняем в столбцах, что-то на что-то
def chang_column(column, to_replace, value):
    changed_column = column.str.replace(to_replace, value)
    return changed_column

df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y')
df[to_replace] = df[to_replace].apply(chang_column, to_replace=',', value='.', axis=0)
df[to_replace] = df[to_replace].apply(pd.to_numeric, downcast='float', errors='coerce', axis=0)

df.fillna(0,inplace = True)

pd.options.display.float_format = '{:,.2f}'.format

## Топ 10 запросов по конверсии 

In [4]:
key_word_column = 'Поисковый запрос'

data_table = pd.pivot_table(df, index = key_word_column,
               aggfunc = column_to_aggregate).sort_values('Конверсии', 
                                                          ascending=False).reset_index()

top10_table = data_table.head(10).copy()
top10_table['Цена конверсии'] = top10_table['Расход (руб.)'] / top10_table['Конверсии']
top10_table

,Поисковый запрос,Клики,Конверсии,Показы,Расход (руб.),Цена конверсии
0,автошкола ростов на дону,201,18.00,690,"8,535.85",474.21
1,автошкола ростов на дону цены,53,7.00,181,"1,920.13",274.30
2,автошкола ростов,110,6.00,359,"4,568.93",761.49
3,мотошкола ростов на дону,24,5.00,108,394.63,78.93
4,автошколы ростова на дону,68,5.00,267,"2,078.16",415.63
5,автошкола в ростове на дону,39,4.00,135,"1,121.75",280.44
6,автошкола,139,4.00,846,"6,460.61","1,615.15"
7,автошкола на еременко ростов,1,3.00,1,19.77,6.59
8,автошколы в ростове на дону,33,3.00,160,"1,371.31",457.10
9,автошкола суворовский район ростов,3,3.00,11,55.46,18.49


## Расход на запросы которые дают нулевые конверсии

In [5]:
sr_zero_conversion = sum((data_table[data_table['Конверсии'] < 1]['Расход (руб.)']))
sr = sum(data_table['Расход (руб.)'])

print("Расход на запросы с нулевой конверсией: {:.2f}\n\
Расход всего на запросы: {:.2f}\n\
Процент расходуемых в пустую средств: {:.2%}"
      .format(sr_zero_conversion, sr,sr_zero_conversion/sr))

Расход на запросы с нулевой конверсией: 49538.63
Расход всего на запросы: 82840.19
Процент расходуемых в пустую средств: 59.80%


### Какое количество запросов, сколько приносит конверсий, какой расход, какая цена конверсии.

In [6]:
conversion_top = pd.pivot_table(data_table, index = 'Конверсии', aggfunc = {'Поисковый запрос':'count',
                                                           'Расход (руб.)':'sum'}).reset_index()
conversion_top['Цена конверсии'] = \
conversion_top['Расход (руб.)'] / \
(conversion_top['Конверсии'] * conversion_top['Поисковый запрос'])
conversion_top

,Конверсии,Поисковый запрос,Расход (руб.),Цена конверсии
0,0.00,11383,"49,538.63",inf
1,1.00,127,"5,396.75",42.49
2,2.00,14,"1,326.84",47.39
3,3.00,5,"1,497.91",99.86
4,4.00,2,"7,582.36",947.79
5,5.00,2,"2,472.79",247.28
6,6.00,1,"4,568.93",761.49
7,7.00,1,"1,920.13",274.30
8,18.00,1,"8,535.85",474.21


## Тут мы нормируем запросы чтобы исключить их склонения.

In [7]:
data_dict = data_table.reset_index().to_dict('records')
# Получаем Леммы
def get_lemm_list_dict(list_dict,key_word):
    for i in range(len(list_dict)):
        #list_dict[i]['Лемма'] = m.lemmatize(list_dict[i][key_word])[:-1]
        lemma = list(set(m.lemmatize(list_dict[i][key_word])[:-1]))
        lemma.sort()
        list_dict[i]['Лемма'] = [item.replace(' ','') for item in lemma if item != ' ']

def get_lemm(list_dict,key_word):
    for item in list_dict:
        lemma = m.lemmatize(item[key_word])[:-1]
        del_char = ' '
        while del_char in lemma: 
            lemma.remove(del_char)
        item['Лемма'] = set(lemma)
#–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

# Определяет вложенность вернет true если вложен
def inside_phrase(text, intext, fl = True):
    if len(text) > len(intext):
        fl = False
    else:
        for word in text:
            if word not in intext:
                fl = False
                break
    return fl

def inside_phrase2(text, intext, fl = True):
    if text.issubset(intext):
        return fl
    else:
        fl = False
    return fl
#set(list_1).issubset(list_2) # Входит ли list_1 в list_2 ?
#–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

# Создает словарь на основе словаря с колонками для группировки
# и добавляет в него ключ 'Кол-во вложений'
def column_to_accumulate(dict_column):
    new_dict = {}
    for item in dict_column:
        new_dict[item] = 0
    return new_dict

def rename_dict_key(rename_dict):
    new_dict = {}
    for item in rename_dict:
        new_dict[item + '_всего'] = rename_dict[item]
    return new_dict
    
# Определим сколько раз фраза встречается в списке фраз
# Аргумент Словарь
# Вернет список [фраза, [лемма, как, спискок], количество раз]
def count_phrases_q(keywords_lemm_list,column):
#   ––– BEGIN Прогресс бар –––
    iterator = 0
    len_list_dict = len(keywords_lemm_list)
#   ––– END Прогресс бар –––

    keywords_lemm_count_list = []
    for text in keywords_lemm_list:
        c = 0
        a = column_to_accumulate(column)
        dother_id_word = [] 
        for intext in keywords_lemm_list:
            if inside_phrase2(text['Лемма'], intext['Лемма']):
                for i in a:
                    a[i] = float(a[i]) + float(intext[i])
                c += 1
                dother_id_word.append(intext['index'])
        b = rename_dict_key(a)
        b['Вложений'] = c
        b['Дочерние ID'] = dother_id_word
        b['Лемматизированный запрос'] = lemm_to_string(text['Лемма'])
        text.update(b)
        keywords_lemm_count_list.append(text)
        
#   ––– BEGIN Прогресс бар –––
        iterator += 1
        print(F'\rВложения: {iterator} / {len_list_dict} ', end='', flush=False)
#   ––– END Прогресс бар –––
    return keywords_lemm_count_list

# Преобразует список в строку
def lemm_to_string(lemm_list):
    return ' '.join(lemm_list)

In [8]:
# get_lemm_list_dict(data_listq,key_word_column)
#Основное решение
data_listq = copy.deepcopy(data_dict[:])
get_lemm(data_listq,key_word_column)
x = count_phrases_q(data_listq,column_to_aggregate)

Вложения: 11536 / 11536 

### Получим уникальные слова

In [10]:
key_list = []
for item in x:
    key_list += item['Лемма']
key_list = list(set(key_list))

In [11]:
# Количесво вложений слов
def words_count(key_list,data_listq,column_to_aggregate):
    out_list = []
    for item_list in key_list:
        c = 0
        a = dict.fromkeys(column_to_aggregate, 0)
        for item_dict in data_listq:
            if set(item_list.split()).issubset(item_dict['Лемма']):
                #inside_phrase(item_list.split(), item_dict['Лемма']):
                c +=1
                for i in a:
                    a[i] += float(item_dict[i])
        a['Слово'] = item_list
        a['Вложений'] = c
        out_list.append(a)
    return out_list
xx = words_count(key_list,data_listq,column_to_aggregate)
# xx - очень важная переменная

## Топ 10 слов по конверсии

In [12]:
words_table = pd.DataFrame(xx).sort_values(by = 'Конверсии', ascending = False)
words_table.head(10)

,Показы,Клики,Конверсии,Расход (руб.),Слово,Вложений
1498,"9,325.00","1,798.00",151.00,"50,133.78",ростов,3238
1706,"11,767.00","1,884.00",143.00,"48,002.16",на,5412
5683,"10,941.00","1,795.00",129.00,"56,390.46",автошкола,4322
2370,"6,818.00","1,357.00",116.00,"37,865.96",дон,2166
2328,"6,773.00",963.00,62.00,"22,562.79",в,3628
1833,"5,443.00",587.00,49.00,"11,077.07",категория,2598
3597,"2,969.00",488.00,34.00,"11,038.82",обучение,1626
688,"5,725.00",659.00,34.00,"12,202.60",право,3519
4433,"1,209.00",291.00,27.00,"7,232.18",цена,434
3772,380.00,88.00,19.00,"2,071.98",район,185


### Слова которые можно использовать в качестве минус-слов (Топ 20)
Теоретически если их исключить то можно экономить бюджет не боясь просадки конверсий.  
сортировка по **Расход (руб.)**

In [13]:
words_table[words_table['Конверсии'] < 1].sort_values(by='Расход (руб.)', ascending = False).head(20)

,Показы,Клики,Конверсии,Расход (руб.),Слово,Вложений
4537,846.00,53.00,0.00,"1,092.74",машина,518
161,586.00,36.00,0.00,956.63,по,389
1561,649.00,35.00,0.00,783.21,какой,491
3474,117.00,17.00,0.00,556.82,рейтинг,30
2245,99.00,18.00,0.00,555.48,хороший,60
1279,215.00,27.00,0.00,499.99,а1,122
2168,122.00,18.00,0.00,434.09,учеба,81
2041,153.00,17.00,0.00,422.44,легковой,101
4021,338.00,25.00,0.00,412.50,сколька,213
3926,522.00,17.00,0.00,392.60,ли,402


### Тут собраны Топ 30 спектров с расходами и конверсиями
если разобрать эти спектры, можно не плохо так сэкономить, но чистка может занять много вермени.

In [14]:
t = pd.DataFrame(x)
top10_spektr = t[['Лемматизированный запрос',
   'Показы_всего',
   'Клики_всего',
   'Конверсии_всего',
   'Расход (руб.)_всего',
   'Вложений']].drop_duplicates().sort_values(by = 'Конверсии_всего', ascending = False).head(30)
top10_spektr['Цена конверсии всего'] = top10_spektr['Расход (руб.)_всего'] / top10_spektr['Конверсии_всего']
top10_spektr

,Лемматизированный запрос,Показы_всего,Клики_всего,Конверсии_всего,Расход (руб.)_всего,Вложений,Цена конверсии всего
4876,ростов,"9,325.00","1,798.00",151.00,"50,133.78",3238,332.01
6,автошкола,"10,941.00","1,795.00",129.00,"56,390.46",4322,437.14
4909,дон ростов на,"6,760.00","1,349.00",116.00,"37,696.94",2131,324.97
2,ростов автошкола,"5,705.00","1,229.00",108.00,"39,129.82",1510,362.31
8584,на автошкола,"5,244.00","1,064.00",92.00,"32,690.43",1655,355.33
0,дон ростов на автошкола,"4,286.00",938.00,83.00,"29,881.00",1037,360.01
6952,в ростов,"3,845.00",715.00,54.00,"16,876.51",1580,312.53
6953,в на ростов,"3,271.00",631.00,50.00,"14,888.78",1276,297.78
8833,категория,"5,443.00",587.00,49.00,"11,077.07",2598,226.06
319,право,"5,725.00",659.00,34.00,"12,202.60",3519,358.90


In [15]:
tt = t[['Лемматизированный запрос','Конверсии','Расход (руб.)']]
pd.pivot_table(tt, index = 'Конверсии', aggfunc = {'Лемматизированный запрос':'count','Расход (руб.)':'sum'},margins = True)

,Лемматизированный запрос,Расход (руб.)
Конверсии,,
0.0,11383,"49,538.63"
1.0,127,"5,396.75"
2.0,14,"1,326.84"
3.0,5,"1,497.91"
4.0,2,"7,582.36"
5.0,2,"2,472.79"
6.0,1,"4,568.93"
7.0,1,"1,920.13"
18.0,1,"8,535.85"


In [16]:
# Оставляет фразы содержащие слова из списка
del_list = [['автошкола', 'на', 'в'],['еременко'],['суворовский']]
out_list =[]
for intext in x:
    fl = False
    for text in del_list:
        if inside_phrase(text, intext['Лемма']):
            fl = True
            break# Если False то слова из фразы есть в списке минус слов и мы не добавляем
    if fl:
        out_list.append(intext)        
pd.DataFrame(out_list)

,index,Поисковый запрос,Клики,Конверсии,Показы,Расход (руб.),Лемма,Показы_всего,Клики_всего,Конверсии_всего,Расход (руб.)_всего,Вложений,Дочерние ID,Лемматизированный запрос
0,5,автошкола в ростове на дону,39,4.00,135,"1,121.75","{на, автошкола, в, дон, ростов}","1,550.00",330.00,27.00,"9,337.61",472,"[5, 8, 18, 25, 32, 37, 40, 46, 47, 48, 50, 55,...",на автошкола в дон ростов
1,7,автошкола на еременко ростов,1,3.00,1,19.77,"{еременко, на, ростов, автошкола}",7.00,3.00,3.00,48.01,4,"[7, 4813, 6346, 8089]",еременко на ростов автошкола
2,8,автошколы в ростове на дону,33,3.00,160,"1,371.31","{на, автошкола, в, дон, ростов}","1,550.00",330.00,27.00,"9,337.61",472,"[5, 8, 18, 25, 32, 37, 40, 46, 47, 48, 50, 55,...",на автошкола в дон ростов
3,9,автошкола суворовский район ростов,3,3.00,11,55.46,"{район, ростов, суворовский, автошкола}",13.00,5.00,3.00,131.19,2,"[9, 5852]",район ростов суворовский автошкола
4,18,сколько стоит обучение в автошколе в ростове н...,5,2.00,20,148.10,"{на, автошкола, 2020, в, дон, сколько, стоить,...",27.00,6.00,3.00,177.56,5,"[18, 50, 5643, 5647, 5770]",на автошкола 2020 в дон сколько стоить обучени...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782,11289,замена водительского удостоверения по окончани...,0,0.00,1,0.00,"{замена, на, срок, по, автошкола, в, удостовер...",1.00,0.00,0.00,0.00,1,[11289],замена на срок по автошкола в удостоверение ок...
783,11346,записаться в автошколу в группу на права,0,0.00,1,0.00,"{на, автошкола, в, право, записываться, группа}",1.00,0.00,0.00,0.00,1,[11346],на автошкола в право записываться группа
784,11426,как ездить в автошколе научиться на площадке,0,0.00,1,0.00,"{на, площадка, автошкола, в, научаться, ездить...",1.00,0.00,0.00,0.00,1,[11426],на площадка автошкола в научаться ездить как
785,11438,как записаться на вождение в автошколе,0,0.00,1,0.00,"{на, автошкола, в, записываться, вождение, как}",2.00,0.00,0.00,0.00,2,"[9247, 11438]",на автошкола в записываться вождение как
